In [9]:
import pandas as pd

ds = pd.read_csv("../data/small_dataset.txt", header=None, names=["item", "user", "ranking"]).iloc[:5000]
#ds.iloc[:5000].to_csv()

In [2]:
ds.head()

,item,user,ranking
0,1,1488844,3
1,1,822109,5
2,1,885013,4
3,1,30878,4
4,1,823519,3


In [45]:
len(set(ds['user']))

4829

In [10]:
# Make necessary imports
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.mllib.random import RandomRDDs
from pyspark.sql import SQLContext
from pyspark import sql
import numpy as np

In [35]:
def normalization(x):
    s = sum(x[1])
    s_last = 0
    x_new = []
    for i in range(len(x[1])-1):
        x_new.append(x[1][i] / s)
        s_last += x_new[i]
    x_new.append(1 - s_last)
    return (x[0], np.array(x_new))

def create_uniform_rdd_rowindexed_matrix(sc, nrow, ncol, normalize=False):
    rdd = []
    for i in range(nrow):
        rdd.append((i, np.random.rand(ncol)))
    #print(rdd)
    rdd = sc.parallelize(rdd)
    if normalize:
        rdd = rdd.map(normalization)
    return rdd

In [63]:
def update_h(hxy_tuple):
    hx, y = hxy_tuple
    h, x = hx
    d = h @ y # a scalar(D_ii)
    e = h @ x.reshape(len(x[0]))
    h_new = h * np.divide(x+d, y+e) # broadcase add
    return h_new

In [64]:
def compute_p_i_R(ratings, sum_ratings, item_probs, ld):
    """Implement the function to compute the probability of items i, given the relevance context of users u.
    Gets as input information on ratings and total ratings for all users of a cluster, then implements multiple
    loops to calculate required outputs."""
    
    # From broadcast values
    sum_ratings = dict(sum_ratings)
    item_probs = dict(item_probs)
    
    # Dictionary with nexted dictionaries. Stores for each user,
    # all the items and their p(i|u) item user probability
    user_items = dict()
    
    for user, item, rating in ratings:
        
        # Find sum of ratings for user and items
        total_ratings = sum_ratings[user]
        total_items = item_probs[item]
        
        # Calculate probability p(i|u), and stores it in user_items nested dictionary
        prob = ((1 - ld) * rating / total_ratings) + ld * total_items
            
        if user in user_items:
            user_items[user][item] = prob
        else:
            user_items[user] = dict()
            user_items[user][item] = prob
    
    # Find the set of all user item pairs
    user_item_pairs = set()
    for user in user_items.keys():
        for item in item_probs.keys():
            user_item_pairs.add((user, item))
    
    out = dict()
    # For every user item pair (i.e. every cell in prediction matrix A)
    for user, item in user_item_pairs:
        
        prod = 1
        # Loop over all items that have been assigned a probability by this user
        for item_t in user_items[user].keys():
            
            sm = 0
            # Loop over all users in the neighbourhood (cluster)
            for user_j in user_items.keys():
                
                # Compute the formula for this particular (item, user)
                p_item_j = user_items[user_j].get(item, 0)
                p_t_j = user_items[user_j].get(item_t, 0)
                
                # Sum over the inner loop
                sm += p_item_j * p_t_j
        
        # Take a product over the outer loop        
        prod *= sm
        
        # Store the (item, user) probability
        out[(item, user)] = prod
    
    return out

In [65]:
class MRAlgorithm:
    
    def __init__(self, datapath, k, ld):
        self.k = k
        self.ld = ld
        self.n = 2649429
        self.m = 6
        self.sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]").set("spark.driver.memory", "15g"))
        self.input = self.sc.textFile(datapath).map(lambda x: (int(x.split(',')[0])-1,
                                                               int(x.split(',')[1])-1,int(x.split(',')[2])))
        self.A = self.input
        self.H = create_uniform_rdd_rowindexed_matrix(self.sc, self.n, self.k, normalize=True)
        self.W = create_uniform_rdd_rowindexed_matrix(self.sc, self.m, self.k)
        
    def compute_H(self):
        print("Compute H")
    
        ## compute X
        # M1
        W_b = self.sc.broadcast(self.W.collect())
        Aw = self.A.map(lambda x: (x[1], x[2]*W_b.value[x[0]][1])) #Wi.T
        #print(Aw.collect())
        # R1 
        X = Aw.reduceByKey(lambda x, y: x+y) # add a combiner? #.map(lambda x: (x[0], x[1]))
    
        
        ## compute B
        # M2
        ww = self.W.map(lambda x: (None, np.outer(x[1],x[1]))) # w.T.dot(w)? #import numpy.linalg
        # R2
        B = ww.reduceByKey(lambda x, y: x+y).map(lambda x: x[1]) # without map?
        
        ## compute Y
        # M3
        B_b = self.sc.broadcast(B.collect())
        y = self.H.map(lambda x: (x[0], B_b.value@x[1]))
        
        ## update H
        # M4
        self.hxy = self.H.join(y).join(X)
        #print(self.hxj.collect())
        # R4
        # why is it a reduce phase instead of a map phase?
        hnew = self.hxy.map(lambda x: (x[0], update_h(x[1])))
        self.H = hnew
        
    def compute_W(self):
        print("Compute W")
        #M5: Compute matrix S (repartition join of H and A)
        tmp_M5 = self.H.join(self.A.map(lambda x: (x[1], (x[0], x[2]))))

        #R5: Emit each hj according to row i
        self.tmp_M5 = tmp_M5.map(lambda x: (x[1][1][0], x[1][1][1] * x[1][0]))

        
        
        #M6: Finish the computation of matrix S -- identity mapper so no code needed
        #R6: Emit each row in matrix S
        self.S = self.tmp_M5.reduceByKey(lambda x, y : x +y)

        
        #M7: Calculate matrix C (same as calculating B in compute_H)
        tmp_M7 = self.H.map(lambda x: (0, np.outer(x[1], x[1])))
        #R7: Emit matrix C from the sum of matrices
        C = tmp_M7.reduceByKey(lambda x, y : x + y).map(lambda x: x[1])
        
        #M8: Compute matrix T
        C_b = self.sc.broadcast(C.collect())
        self.T = self.W.map(lambda x: (x[0], x[1]@C_b.value))
        

        #M9: update W with broadcast join of S and T. W = W * S/T
        T_b = self.sc.broadcast(self.T.collectAsMap())
        S_b = self.sc.broadcast(self.S.collectAsMap())
        self.W = self.W.map(lambda x: (x[0], x[1]*np.divide(S_b.value[x[0]], T_b.value[x[0]])))
        
    def iterate_HW(self): # , iteration_time=25, epsilon
        i = 0
        while i < 5: # or np.norm(previous_H, H) < epsilon
            self.compute_H()
            self.compute_W()
            i += 1
            
            
    def assign_clusters(self):
        print("Assign clusters")
        #M10: Map user to cluster with highest probability
        self.clusters = self.H.map(lambda x: (x[0], np.where(x[1][0] == max(x[1][0]))[0][0]))
        
        #M11: Emit a 1 for each user that is in a cluster
        self.clustersizes = self.clusters.map(lambda x: (x[1], 1))
        
        #R11: Count the number of users per cluster
        self.clustersizes = self.clustersizes.reduceByKey(lambda x,y: x+y)
        
    def RM2_distribution(self):
        print("Compute RM2_distribution")
        #M12: Emit each rating that a user gave
        self.ratings = self.input.map(lambda x: (x[1], x[2]))
        
        #R12: Sum ratings given by user
        self.ratings = self.ratings.reduceByKey(lambda x, y: x+y)
        
        #Accumulator containing total count of ratings
        globalcount = self.sc.accumulator(0)
        self.ratings.foreach(lambda x: globalcount.add(x[1]))
        
        #M13: Emit each rating that an item received
        items = self.input.map(lambda x: (x[0], x[2]))
        
        #Broadcast the total count of ratings, to be used in next reduce
        globalcount = self.sc.broadcast(globalcount.value)
        
        #R13: Compute the probability of item in the collection
        items = items.reduceByKey(lambda x,y: x+y)
        
        #Divide previous values by the total count of ratings, to normalize probabilities
        items = items.map(lambda x: (x[0], x[1]/globalcount.value))
        
        #Broadcast all users and which cluster they are in
        clusters = self.sc.broadcast(self.clusters.collectAsMap())
        
        #M14-a: Map each rating from input to the cluster assigned to user
        self.input2cluster = self.input.map(lambda x: (clusters.value[x[1]], (x[1], x[0], x[2]))).groupByKey()
        
        #M14-b: Map sum of ratings of a user to their assigned cluster
        self.ratings2cluster = self.ratings.map(lambda x: (clusters.value[x[0]], (x[0], x[1]))).groupByKey()
        
        #Repartition join these two maps by cluster
        self.fullClusters = self.input2cluster.join(self.ratings2cluster)
        
        #Broadcast probabilities of each item
        items_bc = self.sc.broadcast(items.collectAsMap())
        
        ld_bc = self.sc.broadcast(self.ld)
        
        #Final reduce combines all information from above, needs W and H
        self.out = self.fullClusters.flatMap(lambda x: (x[0], compute_p_i_R(x[1][0], x[1][1], items_bc.value, ld_bc.value)))
        

In [68]:
#Algorithm = MRAlgorithm('../data/small_dataset.txt', k=5)
#Algorithm = MRAlgorithm('../data/smaller_dataset.txt', k=5, ld=0.5)
#Algorithm.iterate_HW()
# Algorithm.compute_W()
# Algorithm.assign_clusters()
# Algorithm.RM2_distribution()

In [12]:
clusters = Algorithm.out.collect()

In [83]:
Algorithm.input.take(10)

 ## Experiments and results

 #### evaluation of the recommendation algorithm

In [139]:
# split training and testing data
data = pd.read_csv("../data/prepared_data_1.txt", header=None, names=["item", "user", "ranking"])
from sklearn.model_selection import train_test_split
train, test = train_test_split(data) # by default 0.33

In [141]:
test.head()

,item,user,ranking
731970,191,849362,4
9677405,1902,1041130,2
2258526,427,1115506,4
19287790,3650,2443405,4
23596765,4415,1686279,5


In [173]:
def generate_Lu_k(clusters, user_id, k):
    # generate a list of k (item, ranking) pairs sorted on ranking for the user 𝑢
    Lu = [] # list of all relevant items
    # iterate over clusters
    for i in range(len(clusters)//2):
        relevant_users = [x[1] for x in clusters[2*i+1].keys()]
        if user_id in relevant_users:
            Lu += [(x[0][0], x[1]) for x in list(clusters[2*i+1].items())]
    Lu_unique = list(set(Lu))
    Lu_unique.sort(key=lambda x:x[1], reverse=True) 
    return Lu_unique[:k]

def top_k_items(Lu_k):
    return [item_ranking_tuple[0] for item_ranking_tuple in Lu_k]
    
def generate_Ru(df, user_id):
    # generate a list of ranking for all the items having a test rating by some user and no training rating by 𝑢
    return list(df[df["user"] == user_id]["item"])

def intersection_proportion(Luk, Ru, k):
    return len(set(Luk) & set(Ru)) / k

def precision(k, users):
    proportions = np.array([intersection_proportion(generate_Lu_k(clusters, i, k), generate_Ru(df, i, k), 10) for i in users])
    return np.mean(proportions)
    

In [174]:
Luk = generate_Lu_k(clusters, 766488, 10)
res = top_k_items(Luk)
print(Luk)
print(res)

[(2, 356.3258262852286), (0, 145.04464799243766), (5, 66.07494276928142), (4, 37.813576458711005), (3, 11.27704628851279), (1, 2.381702320505256), (2, 1.9098670197560526), (0, 0.7033949162594025), (0, 0.2906373760510468), (4, 0.2316043017684758)]
[2, 0, 5, 4, 3, 1, 2, 0, 0, 4]


In [ ]:
# test
# Ru is generated from the whole dataset while Luk is only generated from the train subset

 #### analysis of the scalability

 #### Effectiveness experiments